In [6]:
import cv2
import time
import numpy as np
import scipy
from scipy import signal

def waste_frames(cap,number):
    for i in range(number):
        cap.read()
        
def waste_time(cap):
    waste_frames(cap,0)
    

def getTotalFrames(input_source):
    tempCap = cv2.VideoCapture(input_source)
    ret=0
    while 1:
        hasFrame, frame=tempCap.read()
        if not hasFrame:
            break
        ret+=1
    return ret

protoFile = "hand/pose_deploy.prototxt"
weightsFile = "hand/pose_iter_102000.caffemodel"
nPoints = 21
POSE_PAIRS = [ [0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20] ]

threshold = 0.2

In [ ]:
toSave = 2
fileName = "tempUser.txt"
saveString = prompt("Do you want to create a new user or edit old user?(Y/N):")
if saveString == 'Y' or saveString == 'y' or saveString == 'yes' or saveString == 'Yes':
    toSave = 1
    fileName = prompt("Enter the unique username:")+".txt"
elif saveString == 'N' or saveString == 'n' or saveString == 'no' or saveString == 'No':
    toSave = 0
else
    print("Can't take a defined input. Will give output as a list of list of tuples")

In [8]:
input_source = "test-1.webm"

totalFrames=getTotalFrames(input_source)
print("Total frames: {}".format(totalFrames))

Total frames: 148


In [10]:
cap = cv2.VideoCapture(input_source)
waste_time(cap)
hasFrame, frame = cap.read()

frameWidth = frame.shape[1]
frameHeight = frame.shape[0]

aspect_ratio = frameWidth/frameHeight

inHeight = 368
inWidth = int(((aspect_ratio*inHeight)*8)//8)

vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
k = 0

smoothed_data, data, previous_points, frame_number = [], [], [], 0
for i in range(nPoints): previous_points.append((0,0))
# Smoothing parameters - choose a window length of about 1/2 to 1/4 the fps (so 60 fps --> window length of 33)
window_length, exponent_value = 5,2 # Window length must be odd

In [5]:
### This is going to take some time before the party get's started! The downside of smoothing is that
### data from the past and the future is required to smooth data in the present. This means that all the frames
### in the video must be processed before smoothing the data and displaying the result. This method is therefore
### not suitable for realtime results.
frameSkip = 0
inStart = 1
while 1:
    k+=1
    t = time.time()
    waste_time(cap)
    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)
    if not hasFrame:
        break

    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()

    print("forward = {}".format(time.time() - t))

    # Empty list to store the detected keypoints
    points = []
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 6, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            # Add the last known point (ex: if thumb is not detected, use thumb position from previous frame)
            points.append(previous_points[i])
            
    # Save the data from the model - data is a list of lists. Each element is a list containing the 22 coordinates for the hand.
    data.append(points)
    
    previous_points = points

    print("total = {}".format(time.time() - t))

forward = 4.519621849060059
total = 4.594968557357788
forward = 3.785804271697998
total = 3.803959846496582
forward = 3.8616249561309814
total = 3.882234573364258
forward = 3.576322555541992
total = 3.5945487022399902
forward = 3.427830457687378
total = 3.4459617137908936
forward = 3.9876701831817627
total = 4.013535737991333
forward = 3.7925262451171875
total = 3.813220977783203
forward = 3.5775961875915527
total = 3.5962862968444824
forward = 3.692706346511841
total = 3.7114877700805664
forward = 3.728640556335449
total = 3.7468299865722656
forward = 3.3882241249084473
total = 3.405975580215454
forward = 3.517738103866577
total = 3.5378127098083496
forward = 3.6042373180389404
total = 3.656322479248047
forward = 3.3203461170196533
total = 3.338383674621582
forward = 3.8526711463928223
total = 3.8709444999694824
forward = 3.6266560554504395
total = 3.647129774093628
forward = 3.7671220302581787
total = 3.7912166118621826
forward = 4.298116207122803
total = 4.324123382568359
forward = 

In [4]:
frameSkip=0
data3 = []
for i in data:
    if (0,0) in i:
        frameSkip+=1
    else:
        break
data3 = data[frameSkip:]

Data3: [[(335, 318), (397, 318), (459, 297), (500, 266), (541, 234), (449, 203), (500, 151), (521, 151), (552, 151), (417, 192), (449, 150), (500, 130), (511, 141), (386, 193), (397, 140), (448, 119), (479, 109), (346, 203), (335, 151), (335, 119), (335, 78)], [(325, 318), (397, 317), (459, 297), (500, 255), (531, 224), (438, 214), (489, 161), (520, 151), (541, 162), (407, 203), (438, 140), (479, 119), (490, 109), (376, 193), (387, 130), (427, 109), (469, 98), (345, 203), (335, 151), (335, 109), (335, 68)], [(325, 318), (387, 307), (458, 287), (490, 255), (521, 213), (438, 213), (490, 161), (521, 171), (531, 192), (407, 193), (428, 130), (469, 109), (500, 119), (376, 192), (386, 130), (418, 98), (459, 88), (345, 193), (325, 140), (325, 109), (325, 67)], [(315, 318), (387, 307), (449, 286), (490, 245), (510, 213), (438, 213), (480, 161), (511, 172), (521, 193), (397, 192), (418, 120), (459, 99), (490, 109), (366, 182), (376, 130), (417, 88), (449, 78), (335, 193), (324, 150), (324, 109)

In [ ]:
# Re-capture the source, so that the video starts at the beginning again
data=data3
cap = cv2.VideoCapture(input_source)
nPoints=21
waste_frames(cap,frameSkip)

# Smooth it out
# Split the data so that just the x values for the first point are made into a list
smoothed_data_in_series = []
for point_index in range(nPoints): # Iterate through each point (wrist, thumb, etc...)
    data_point_series_x = []
    data_point_series_y = []
    for values in data: # Iterate through the series of data (each frame of video)
        data_point_series_x.append(values[point_index][0])
        data_point_series_y.append(values[point_index][1])
    # Smooth the x and y values
    smoothed_data_point_series_x = signal.savgol_filter(data_point_series_x, window_length, exponent_value)
    smoothed_data_point_series_y = signal.savgol_filter(data_point_series_y, window_length, exponent_value)
    smoothed_data_in_series.append(smoothed_data_point_series_x)
    smoothed_data_in_series.append(smoothed_data_point_series_y)
    
# Now the data is sepearted into 44 lists (two lists for each of the 22 locations of the hand, time to zip()
for current_frame_number in range(len(smoothed_data_in_series[0])):
    frame_values = []
    for point_index in range(nPoints):
        x = smoothed_data_in_series[point_index*2][current_frame_number]
        y = smoothed_data_in_series[point_index*2+1][current_frame_number]
        frame_values.append((x,y))
    smoothed_data.append(frame_values)

# Iterate through each frame of data
for values in smoothed_data:
    waste_time(cap)
    hasFrame, img = cap.read()

    # When data is smoothed, floats are introduced, these must be eliminated becase cv2.circle requries integers
    values = np.array(values, int)  
    
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        cv2.line(img, (values[partA][0], values[partA][1]),(values[partB][0], values[partB][1]), (32, 255, 0), 2, lineType=cv2.LINE_AA)
        cv2.circle(img, (values[partA][0], values[partA][1]), 2, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.circle(img, (values[partB][0], values[partB][1]), 2, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    cv2.imshow('Output-Skeleton', img)
    cv2.waitKey(0)
    vid_writer.write(img)

cv2.destroyAllWindows()
vid_writer.release()
cap.release()
